<a href="https://colab.research.google.com/github/Santosdevbjj/analiseRiscosAtrasoObras/blob/main/Notebooks/02_limpeza_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# ============================
# Notebook 02 — Limpeza e Pré‑Processamento (ETL)
# ============================
import os
import pandas as pd
import numpy as np

raw_path = "data/raw/"

# Carregar
obras = pd.read_csv(raw_path + "obrasccbjj.csv")
forn = pd.read_csv(raw_path + "fornecedoresccbjj.csv")
clima = pd.read_csv(raw_path + "climaccbjj.csv")
mao = pd.read_csv(raw_path + "mao_obraccbjj.csv")
ativ = pd.read_csv(raw_path + "atividadesccbjj.csv")
supr = pd.read_csv(raw_path + "suprimentosccbjj.csv")
bot = pd.read_csv(raw_path + "base_consulta_botccbjj.csv")
rel = pd.read_csv(raw_path + "relatorio_consolidadoccbjj.csv")

# Padronização de texto
for df in [obras, clima, mao, ativ, supr, bot, rel]:
    for col in df.columns:
        if df[col].dtype == object:
            df[col] = df[col].astype(str).str.strip()

# Tipos corretos
obras["data_inicio_prevista"] = pd.to_datetime(obras["data_inicio_prevista"], errors="coerce")
num_cols_obras = ["orcamento_estimado","prazo_previsto_dias","prazo_real_dias","chuva_mm","Atrasou"]
for c in num_cols_obras:
    obras[c] = pd.to_numeric(obras[c], errors="coerce")

# Nulos: medianas
for c in ["prazo_previsto_dias","prazo_real_dias","chuva_mm","orcamento_estimado"]:
    obras[c] = obras[c].fillna(obras[c].median())

# Definir target
obras["atraso_dias"] = obras["prazo_real_dias"] - obras["prazo_previsto_dias"]
obras["atraso_flag"] = (obras["atraso_dias"] > 0).astype(int)

# Agregações de atividades
agg_ativ = ativ.groupby("id_obra").agg({
    "dias_atraso": "sum",
    "id_atividade": "count"
}).rename(columns={"dias_atraso":"dias_atraso_total","id_atividade":"qtd_atividades"}).reset_index()

# Taxas de insucesso
taxa_obra = supr.groupby("id_obra")["atrasou_entrega"].mean().rename("taxa_insucesso_obra").reset_index()

# Merge principal
merged = obras.rename(columns={"Id_obra":"id_obra"}) \
    .merge(clima, on="id_obra", how="left") \
    .merge(mao.rename(columns={"Id_obra":"id_obra"}), on="id_obra", how="left") \
    .merge(agg_ativ, on="id_obra", how="left") \
    .merge(taxa_obra, on="id_obra", how="left") \
    .merge(bot[["id_obra","complexidade_obra","risco_etapa","nivel_chuva","tipo_solo","material"]], on="id_obra", how="left") \
    .merge(rel[["id_obra","risco_medio","pior_etapa","risco_pior","material_critico","taxa_insucesso"]], on="id_obra", how="left")

# Limpeza de nulos numéricos
num_cols = ["prazo_previsto_dias","prazo_real_dias","chuva_mm","orcamento_estimado","qtd_engenheiros",
            "qtd_pedreiros","qtd_servente_pedreiros","dias_atraso_total","qtd_atividades","taxa_insucesso_obra",
            "complexidade_obra","risco_etapa","nivel_chuva","risco_medio","risco_pior","taxa_insucesso","atraso_dias"]
for c in num_cols:
    if c in merged.columns:
        merged[c] = pd.to_numeric(merged[c], errors="coerce").fillna(merged[c].median())

# Padronizar categóricas
for c in ["cidade","tipo_solo","material","pior_etapa","material_critico"]:
    if c in merged.columns:
        merged[c] = merged[c].astype(str).str.strip().str.title()

# Features derivadas
merged["atraso_pct"] = merged["atraso_dias"] / merged["prazo_previsto_dias"].replace(0, np.nan)
merged["engenheiros_ratio"] = merged["qtd_engenheiros"] / merged["qtd_pedreiros"].replace(0, np.nan)
merged["produtividade_pedreiro"] = merged["prazo_previsto_dias"] / merged["qtd_pedreiros"].replace(0, np.nan)
merged["chuva_intensa_flag"] = (merged["chuva_mm"] >= 200).astype(int)

# Salvar dataset final
os.makedirs("data/processed", exist_ok=True)
out_file = "data/processed/dataset_modelo.csv"
merged.to_csv(out_file, index=False)
print(f"✅ Dataset final salvo: {out_file}")

# Download automático no Colab
try:
    from google.colab import files
    files.download(out_file)
except Exception:
    pass

# Prévia
print(merged[["id_obra","cidade","prazo_previsto_dias","prazo_real_dias","atraso_flag"]].head())

FileNotFoundError: [Errno 2] No such file or directory: 'data/raw/obrasccbjj.csv'